In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype

# try 1

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/future_comes_soon2.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
geometries

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[4:]])
text_geos

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
mp = MultiPolygon(gp.consolidate_polygons([Polygon(ls) for ls in text_geos]))
        

In [ ]:
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)

In [ ]:
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
stp = gp.ScaleTransPrms(n_iters=200, d_buffer=-0.35, d_translate_factor=0.7, angles=30)
stp.d_buffers *= gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=15)

In [ ]:
stp.angles = np.linspace(0, np.radians(360*4), len(stp.d_buffers))

In [ ]:
scaled = gp.scale_trans(dp, **stp.prms)

In [ ]:
layer = gp.merge_LineStrings([p.boundary for p in reversed(scaled)])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(layer)
sk.display(color_mode='None')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0096_future_come_soon.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 2

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/future_comes_soon3.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[4:]])
text_geos

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
mp = so.unary_union(gp.consolidate_polygons([Polygon(ls) for ls in text_geos]))
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
pts = gp.get_random_points_in_polygon(dp, n_points=80)
border_pts = ([drawbox.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=60)])
corners = [Point(x,y) for x in [xmin, xmax] for y in [ymin, ymax]]
text_pts = ([cmp.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=120)])
all_pts = MultiPoint(pts + text_pts + corners +border_pts)

In [ ]:
tris = MultiPolygon(so.triangulate(all_pts))

In [ ]:
bps = []
for p in tris:
    d = p.centroid.distance(drawbox.centroid)
    buffer_dist = np.interp(d, [0, 175], [-0.3, -3])
    bps.append(p.buffer(buffer_dist, join_style=2, cap_style=2))
bgps = MultiPolygon(bps)

In [ ]:
bgpus = so.unary_union([p.buffer(np.random.choice([0., 1], p=[0.8,0.2]), join_style=2, cap_style=2) for p in bgps])

In [ ]:
bps = []
for p in bgpus:
    d = p.centroid.distance(drawbox.centroid)
    buffer_dist = np.interp(d, [0, 175], [-0.1, -0.2])
    bps.append(p.buffer(buffer_dist, join_style=2, cap_style=2))
bgpus = MultiPolygon(bps)

In [ ]:
bg_fills = []
for p in tqdm(bgpus):
    d = p.centroid.distance(drawbox.centroid)
    d_buffer = np.interp(d, [0, 175], [-0.3, -0.55])
    angle = np.interp(d, [0, 175], [np.radians(-30), np.radians(-150)])
    stp = gp.ScaleTransPrms(n_iters=80, d_buffer=d_buffer, 
                            d_translate_factor=0.7,
                            angles=angle)
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=1.5, step_mu=-0.4)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
    scaled = gp.scale_trans(p, **stp.prms, join_style=2, cap_style=2, mitre_limit=30)
    fill = gp.merge_LineStrings([f.boundary for f in reversed(scaled)])
    bg_fills.append(fill)
    
bg = dp.intersection(gp.merge_LineStrings(bg_fills))
bg

In [ ]:
text_hatch = gp.connected_hatchbox(cmp, angle=45, spacing=0.15, dist_thresh=0.25)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(bg)
sk.stroke(2)
sk.geometry(text_hatch)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0097_future_come_soon.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 3

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/future_comes_soon3.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[4:]])
text_geos

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
mp = so.unary_union(gp.consolidate_polygons([Polygon(ls) for ls in text_geos]))
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)
cmp = sa.translate(cmp, yoff=10)
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
pts = gp.get_random_points_in_polygon(dp, n_points=10)
border_pts = ([drawbox.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=10)])
corners = [Point(x,y) for x in [xmin, xmax] for y in [ymin, ymax]]
text_pts = ([cmp.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=30)])
all_pts = MultiPoint(pts + text_pts + corners +border_pts)
tris = MultiPolygon(so.triangulate(all_pts))
tris

In [ ]:
bps = []
for p in tris:
    d = p.centroid.distance(drawbox.centroid)
    buffer_dist = np.interp(d, [0, 175], [-0.3, -3])
    bps.append(p.buffer(buffer_dist, join_style=2, cap_style=2))

cpts = gp.get_random_points_in_polygon(dp, n_points=60)
bps += [c.buffer(np.random.uniform(7, 38)) for c in cpts]


bgps = MultiPolygon(bps)

btbs = so.unary_union([p.boundary.buffer(np.random.uniform(0.1, 2), join_style=2, cap_style=2) for p in bgps]).intersection(drawbox)

dpd = dp.difference(btbs)

In [ ]:
bg_fills = []
for p in tqdm(dpd):
    d = p.centroid.distance(drawbox.centroid)
    d_buffer = np.interp(d, [0, 175], [-0.3, -0.5])
    stp = gp.ScaleTransPrms(n_iters=80, d_buffer=d_buffer, 
                            d_translate_factor=0.5,
                            angles=angle)
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=1.5, step_mu=-0.4)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
    stp.angles = np.linspace(0, np.radians(360*0.5), len(stp.d_buffers))
    scaled = gp.scale_trans(p, **stp.prms, join_style=2, cap_style=2, mitre_limit=30)
    
    fill = gp.merge_LineStrings([f.boundary for f in reversed(scaled)])
    bg_fills.append(fill)
    
layer = dp.intersection(gp.merge_LineStrings(bg_fills))
layer

In [ ]:
stp = gp.ScaleTransPrms(n_iters=200, d_buffer=-0.35, d_translate_factor=0.5, angles=np.radians(-45))
stp.d_buffers *= gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=15)

stp.angles = np.linspace(0, np.radians(360*0.5), len(stp.d_buffers))

scaled = gp.scale_trans(dpd, **stp.prms)

layer = gp.merge_LineStrings([p.boundary for p in reversed(scaled)])

In [ ]:
text_hatch = gp.connected_hatchbox(cmp, angle=90, spacing=0.25, dist_thresh=0.35)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(layer)
sk.stroke(2)
sk.geometry(text_hatch)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0099_future_come_soon.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.05mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 4

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 14 # inches
border:float = 30.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/time2.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[:34]])
stars = MultiLineString([l for l in mls[34:]])

In [ ]:
text_polys = list(text_geos.buffer(4, cap_style=1, join_style=1))

In [ ]:
pbar_start = text_polys[5].centroid.x-15
pbar_end = text_polys[6].centroid.x+15


In [ ]:
pbar = box(minx=pbar_start, miny=610,
           maxx=pbar_end, maxy=670)

In [ ]:

star_polys = [Polygon(ls) for ls in stars]
polys = text_polys + star_polys + [pbar]

In [ ]:
so.unary_union(gp.consolidate_polygons(polys))

In [ ]:
mp = so.unary_union(gp.consolidate_polygons(polys))
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)
cmp = sa.translate(cmp, yoff=10)
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
stp = gp.ScaleTransPrms(n_iters=280, d_buffer=-0.4, 
                            d_translate_factor=0.4,
                            angles=0)
stp.angles = np.linspace(0, np.radians(360*2), len(stp.d_buffers))
db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=9.5, step_mu=-0.)
db_mults[db_mults<0.] = 0.05
stp.d_buffers *= db_mults
scaled = gp.scale_trans(dp, **stp.prms, join_style=2, cap_style=2, mitre_limit=30)
    
fill = gp.merge_LineStrings([f.boundary for f in scaled])

In [ ]:
dists = [-f.hausdorff_distance(dp.centroid) for f in fill]

In [ ]:
sort_fill = gp.merge_LineStrings([fill[i] for i in np.argsort(dists)])

In [ ]:
pbstart = pb.bounds[0]
pbend = pb.bounds[2]
pbwidth = pbend-pbstart

In [ ]:

pbar_fill = box(minx=pbstart+1, miny=pb.bounds[1]+1,
           maxx=pbstart+pbwidth*0.41-1, maxy=pb.bounds[3]-1)

In [ ]:
pbar_hatch = gp.hatchbox(pbar_fill, angle=90, spacing=0.25)

In [ ]:
pbar_hatch_r = MultiLineString([ls for ls in reversed(pbar_hatch)])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(sort_fill)
sk.stroke(2)
sk.geometry(pbar_hatch_r)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0123_progress_bar.svg'

sk.save(savepath, paper_size=(px, py))

# vpype_commands = 'linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm'
# vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

# os.system(vpype_str)

# try 5

In [ ]:
page_x_inches: float = 10 # inches
page_y_inches: float = 7 # inches
border:float = 20.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/time3.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[:34]])
stars = MultiLineString([l for l in mls[34:]])

In [ ]:
pbar_start = text_polys[5].centroid.x-15
pbar_end = text_polys[6].centroid.x+15


In [ ]:
pbary = text_geos[5].centroid.y

In [ ]:
text_polys = list(text_geos.buffer(5, cap_style=1, join_style=1))

In [ ]:
pbar = box(minx=pbar_start, miny=pbary-20,
           maxx=pbar_end, maxy=pbary+40)

In [ ]:

star_polys = [Polygon(ls) for ls in stars]
polys = text_polys + star_polys + [pbar]

In [ ]:
so.unary_union(gp.consolidate_polygons(polys))

In [ ]:
mp = so.unary_union(gp.consolidate_polygons(polys))
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)
cmp = sa.translate(cmp, yoff=10)
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
stp = gp.ScaleTransPrms(n_iters=280, d_buffer=-0.45, 
                            d_translate_factor=0.4,
                            angles=0)
stp.angles = np.linspace(0, np.radians(360*0), len(stp.d_buffers))
db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=9.5, step_mu=-0.)
db_mults[db_mults<0.] = 0.05
stp.d_buffers *= db_mults
scaled = gp.scale_trans(dp, **stp.prms, join_style=2, cap_style=2, mitre_limit=30)
    
fill = gp.merge_LineStrings([f.boundary for f in scaled])

In [ ]:
dists = [-f.hausdorff_distance(dp.centroid) for f in fill]

In [ ]:
sort_fill = gp.merge_LineStrings([fill[i] for i in np.argsort(dists)])

In [ ]:
pb = polys[-1]

In [ ]:
pbstart = pb.bounds[0]
pbend = pb.bounds[2]
pbwidth = pbend-pbstart

In [ ]:

pbar_fill = box(minx=pbstart+1, miny=pb.bounds[1]+1,
           maxx=pbstart+pbwidth*0.41-1, maxy=pb.bounds[3]-1)

In [ ]:
pbar_hatch = gp.hatchbox(pbar_fill, angle=90, spacing=0.25)

In [ ]:
pbar_hatch_r = MultiLineString([ls for ls in reversed(pbar_hatch)])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(sort_fill)
# sk.stroke(2)
# sk.geometry(pbar_hatch_r)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0124_progress_bar.svg'

sk.save(savepath, paper_size=(px, py))

# vpype_commands = 'linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm'
# vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

# os.system(vpype_str)

# try 6

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/relief_comes_soon.svg'
doc = vpype.read_multilayer_svg(svgpath, quantization=1)

In [ ]:
geometries = {}
for k,v in doc.layers.items():
    geometries[k] = v.as_mls()

In [ ]:
mls = geometries[1]
text_geos = MultiLineString([l for l in mls[4:]]).buffer(5, cap_style=2, join_style=2).boundary
text_geos

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
mp = so.unary_union(gp.consolidate_polygons([Polygon(ls) for ls in text_geos]))
smp = gp.scale_like(mp, drawbox)
smp = sa.scale(smp, xfact=0.9, yfact=0.9)
cmp = gp.center_at(smp, drawbox.centroid)
cmp = sa.translate(cmp, yoff=10)
dp = gp.robust_difference(drawbox, cmp)

In [ ]:
pts = gp.get_random_points_in_polygon(dp, n_points=30)
border_pts = ([drawbox.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=10)])
corners = [Point(x,y) for x in [xmin, xmax] for y in [ymin, ymax]]
text_pts = ([cmp.boundary.interpolate(i, normalized=True) for i in np.random.uniform(size=30)])
all_pts = MultiPoint(pts + text_pts + corners +border_pts)
tris = MultiPolygon(so.triangulate(all_pts))
tris

In [ ]:
bps = []
for p in tris:
    d = p.centroid.distance(drawbox.centroid)
    buffer_dist = np.interp(d, [0, 175], [-0.3, -3])
    bps.append(p.buffer(buffer_dist, join_style=2, cap_style=2))

cpts = gp.get_random_points_in_polygon(dp, n_points=60)
bps += [c.buffer(np.random.uniform(7, 38)) for c in cpts]


bgps = MultiPolygon(bps)

btbs = so.unary_union([p.boundary.buffer(np.random.uniform(0.1, 2), join_style=2, cap_style=2) for p in bgps]).intersection(drawbox)

dpd = dp.difference(btbs)

In [ ]:
angle = 45

In [ ]:
bg_fills = []
for p in tqdm(dpd):
    d = p.centroid.distance(drawbox.centroid)
    d_buffer = np.interp(d, [0, 175], [-0.3, -0.5])
    stp = gp.ScaleTransPrms(n_iters=80, d_buffer=d_buffer, 
                            d_translate_factor=0.5,
                            angles=angle)
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=1.5, step_mu=-0.4)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
    stp.angles = np.linspace(0, np.radians(360*0.5), len(stp.d_buffers))
    scaled = gp.scale_trans(p, **stp.prms, join_style=2, cap_style=2, mitre_limit=30)
    
    fill = gp.merge_LineStrings([f.boundary for f in reversed(scaled)])
    bg_fills.append(fill)
    
layer = dp.intersection(gp.merge_LineStrings(bg_fills))
layer

In [ ]:
stp = gp.ScaleTransPrms(n_iters=200, d_buffer=-0.35, d_translate_factor=0.5, angles=np.radians(-45))
stp.d_buffers *= gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=15)

stp.angles = np.linspace(0, np.radians(360*0.5), len(stp.d_buffers))

scaled = gp.scale_trans(dpd, **stp.prms)

layer = gp.merge_LineStrings([p.boundary for p in reversed(scaled)])

In [ ]:
text_hatch = gp.connected_hatchbox(cmp, angle=-90, spacing=0.3, dist_thresh=0.35)

In [ ]:
text_hatch = gp.merge_LineStrings(reversed([ls for ls in text_hatch]))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(layer)
sk.stroke(2)
sk.geometry(text_hatch)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0153_relief_come_soon.svg'

sk.save(savepath)

vpype_commands = 'reloop linemerge --tolerance 0.2mm '
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)